In [ ]:
import json
import requests
from kafka import KafkaProducer
from datetime import datetime

# Variables de configuration
GITHUB_TOKEN = "your_token"  # Remplacez par votre token GitHub
KAFKA_TOPIC = "github_repos"
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"

# Initialisation du producteur Kafka
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Fonction pour rechercher des dépôts GitHub créés dans un mois spécifique
def search_github_repos(month, year):
    query = f"created:{year}-{month:02d}-01..{year}-{month:02d}-30"
    url = f"https://api.github.com/search/repositories?q={query}&sort=created&order=desc&per_page=100"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }

    repos = []
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            repos.extend(data.get("items", []))
            # Vérifiez s'il existe une page suivante
            url = response.links.get('next', {}).get('url', None)
        else:
            print(f"Erreur lors de la récupération des dépôts : {response.status_code}")
            return []

    return repos

# Fonction pour envoyer les noms des dépôts à Kafka
def send_to_kafka(repos):
    for repo in repos:
        repo_data = {
            "repo_name": repo["name"],
            "full_name": repo["full_name"],
            "html_url": repo["html_url"]
        }
        # Envoi des données à Kafka
        producer.send(KAFKA_TOPIC, value=repo_data)
        print(f"Dépôt envoyé à Kafka : {repo['name']}")

# Fonction principale pour récupérer et envoyer les dépôts
def collect_and_send(month, year):
    print(f"Recherche des dépôts GitHub créés en {month}/{year}...")
    repos = search_github_repos(month, year)
    if repos:
        send_to_kafka(repos)
    else:
        print(f"Aucun dépôt trouvé pour {month}/{year}.")

if __name__ == "__main__":
    try:
        # Exemple : récupérer les repos créés en novembre 2024
        collect_and_send(11, 2024)
    except KeyboardInterrupt:
        print("Arrêt du script.")
    finally:
        producer.close()

Recherche des dépôts GitHub créés en 11/2024...
Dépôt envoyé à Kafka : ProxmoxVE
Dépôt envoyé à Kafka : alphafold3
Dépôt envoyé à Kafka : hyperlight
Dépôt envoyé à Kafka : hertz-dev
Dépôt envoyé à Kafka : chonkie
Dépôt envoyé à Kafka : BemiDB
Dépôt envoyé à Kafka : LLM-engineer-handbook
Dépôt envoyé à Kafka : snacks.nvim
Dépôt envoyé à Kafka : someday
Dépôt envoyé à Kafka : project-sid
Dépôt envoyé à Kafka : PhotoDiva-Pro-free-2024
Dépôt envoyé à Kafka : Autodesk-Inventor-free-2024
Dépôt envoyé à Kafka : Exitlag-Free-2024
Dépôt envoyé à Kafka : Canva-Pro-2024
Dépôt envoyé à Kafka : OpenSea-Bidding-Bot-2024
Dépôt envoyé à Kafka : Fl-Studio-2024
Dépôt envoyé à Kafka : AI-Crypto-Signals-Bot
Dépôt envoyé à Kafka : AI-Crypto-Trading-Bot
Dépôt envoyé à Kafka : DimensionX
Dépôt envoyé à Kafka : Slax
Dépôt envoyé à Kafka : web-chains
Dépôt envoyé à Kafka : In-Context-LoRA
Dépôt envoyé à Kafka : Protenix
Dépôt envoyé à Kafka : whirlwind
Dépôt envoyé à Kafka : coding-resources
Dépôt envoyé à Kaf